In [65]:
from scipy.stats import rankdata
import scipy as sp
import numpy as np
import pandas as pd

### 读取数据

In [66]:
df = pd.read_csv('./alldata.csv')

D:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### 辅助函数

In [131]:
def get_data(column):
    df_res = df.pivot(index=['Trddt'], columns='Stkcd', values=[column])
    df_res = df_res[column]
    df_res = df_res.reset_index()
    df_res.fillna(df_res.mean(), inplace=True)
    df_res = df_res.set_index('Trddt')
    def change_extreme(row):
        m=row.mean()
        sd=row.std()
        row=row.apply(lambda x:x if m-3*sd<=x<=m+3*sd else (m+3*sd if x>m+3*sd else m-3*sd))
        return row
    df_res = df_res.apply(change_extreme, axis=1)
    df_res = df_res.apply(lambda row: (row-row.mean())/row.std(), axis=1)
    return df_res
                    

In [103]:
import statsmodels.api as sm 
def compute_IC_t(df,return_nextday):#df为因子值矩阵
    
    #行列对齐
    return_nextday=return_nextday[df.columns.tolist()]
    return_nextday=return_nextday.loc[df.index.tolist()]
    
    #建立新的dataframe，用于存储结果
    result=pd.DataFrame(np.nan,index=df.index,columns=["factor_return","tvalue","IC"])

    #T+1期收益率对T期因子值逐行做回归
    for date in df.index.tolist():
        x=df.loc[date]
        y=return_nextday.loc[date]
        model=sm.OLS(y,x).fit()
        result.loc[date,"factor_return"]=model.params[date]
        result.loc[date,"tvalue"]=model.tvalues[date]
        result.loc[date,"IC"]=return_nextday.loc[date].corr(df.loc[date])
    
    return result

### 计算因子

In [99]:
df_return = df_close / df_close.shift(1) - 1
df_return

Stkcd,1,2,4,5,6,7,8,9,10,11,...,900947,900948,900949,900950,900951,900952,900953,900955,900956,900957
Trddt,,,,,,,,,,,,,,,,,,,,,
2010-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,-0.017292,-0.022642,0.000000,0.035058,-0.028777,0.037518,0.000000,-0.025830,-0.017782,-0.031015,...,0.002387,0.034447,0.012950,0.017305,0.013011,0.010417,0.027816,-0.014184,0.015038,-0.007326
2010-01-06,-0.017167,0.000000,0.000000,-0.019355,0.002778,0.000000,0.000000,-0.000947,0.010345,0.002910,...,-0.001190,-0.005012,0.001420,0.015796,-0.009174,-0.006186,0.020298,0.018705,-0.001235,0.007380
2010-01-07,-0.010917,-0.007722,0.000000,-0.041118,-0.008310,-0.029207,0.000000,0.018957,-0.039249,-0.023211,...,-0.020262,-0.017126,-0.017021,-0.011962,-0.029630,-0.012448,-0.011936,-0.004237,-0.019778,-0.032967
2010-01-08,-0.002208,0.006809,0.000000,0.010292,0.024209,0.007163,0.000000,0.048372,0.017762,0.009901,...,0.007299,0.013552,0.002886,0.007264,0.011450,0.010504,0.006711,0.012766,-0.001261,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-24,-0.008646,0.002773,0.015625,0.004739,-0.011086,0.011696,0.011364,0.019900,-0.019022,-0.007819,...,-0.021127,0.001105,0.000000,0.000000,0.000000,-0.016129,-0.011834,-0.016667,0.000000,-0.026646
2022-01-25,-0.020349,-0.040553,-0.056410,-0.033019,-0.038117,-0.014451,-0.056180,-0.052265,-0.060942,-0.044658,...,-0.025180,-0.026490,0.000000,0.000000,0.000000,-0.004098,-0.014970,-0.016949,0.000000,-0.016103
2022-01-26,-0.011869,-0.004323,0.099802,0.004878,0.009324,-0.049853,0.027778,-0.018382,0.011799,0.024748,...,0.000000,0.007937,0.000000,0.000000,0.000000,0.016461,0.000000,0.008621,0.000000,0.004910


In [110]:
df_result = compute_IC_t(df_open_std.shift(2)[2:], df_return)

### 因子

In [133]:
df_high = get_data('Hiprc')
df_low = get_data('Loprc')
df_open = get_data('Opnprc')
df_close = get_data('Clsprc')
df_volume = get_data('Dnshrtrd')
df_value = get_data('Dnvaltrd')
df_marketcap = get_data('Dsmvosd')
df_low

Stkcd,1,2,4,5,6,7,8,9,10,11,...,900947,900948,900949,900950,900951,900952,900953,900955,900956,900957
Trddt,,,,,,,,,,,,,,,,,,,,,
2010-01-04,-0.073474,-0.604459,-0.270474,-0.794851,-0.583756,-0.754662,-0.687357,-0.596340,-0.556151,-0.603242,...,-1.000831,-0.683214,-1.006596,-1.002090,-1.013010,-1.015405,-1.005621,-1.006149,-1.002536,-1.012604
2010-01-05,-0.110619,-0.619510,-0.269685,-0.796710,-0.600858,-0.756161,-0.686094,-0.608968,-0.565985,-0.619105,...,-0.999496,-0.677982,-1.004930,-1.001362,-1.011377,-1.013729,-1.004119,-1.005011,-1.000875,-1.011175
2010-01-06,-0.114515,-0.622210,-0.272371,-0.790090,-0.601123,-0.747106,-0.688796,-0.615316,-0.567466,-0.625859,...,-1.001845,-0.674723,-1.007523,-1.002900,-1.014011,-1.016322,-1.006022,-1.007806,-1.003346,-1.013848
2010-01-07,-0.123432,-0.616368,-0.268264,-0.795544,-0.601775,-0.750953,-0.684554,-0.614747,-0.580290,-0.625692,...,-0.998191,-0.675715,-1.003542,-0.998434,-1.010352,-1.012257,-1.001434,-1.003258,-0.999529,-1.010190
2010-01-08,-0.122881,-0.616447,-0.265869,-0.797070,-0.597776,-0.752827,-0.682691,-0.601835,-0.582352,-0.623753,...,-0.996972,-0.677331,-1.002249,-0.996972,-1.009027,-1.010894,-1.000016,-1.001640,-0.998393,-1.009108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-24,-0.196563,-0.058285,-0.068324,-0.678432,-0.603626,-0.526877,-0.662240,-0.296629,-0.630180,-0.380179,...,-0.737500,-0.717390,-0.727444,-0.722956,-0.727116,-0.738536,-0.735654,-0.742649,-0.708161,-0.726522
2022-01-25,-0.190111,-0.063019,-0.083444,-0.668653,-0.596029,-0.517893,-0.653415,-0.294833,-0.625208,-0.381722,...,-0.726363,-0.706521,-0.716101,-0.711608,-0.715773,-0.727271,-0.724580,-0.731453,-0.696796,-0.715437
2022-01-26,-0.193306,-0.063293,-0.005510,-0.669693,-0.596151,-0.523593,-0.653278,-0.305591,-0.627012,-0.377820,...,-0.727444,-0.707384,-0.717085,-0.712536,-0.716753,-0.728331,-0.725606,-0.732533,-0.697536,-0.716314
